In [51]:
# Import libraries and functions

# Importing necessary libraries
import yfinance as yf
import pandas as pd

# Importing functions created in the "Financial_Ratios_Metrics.py" file
from Financial_Ratios_Metrics import (calculate_net_profit_margin, calculate_eps, calculate_pe_ratio, 
                                      calculate_dividends_per_share, calculate_dividend_payout_ratio, 
                                      calculate_roe, calculate_roa, calculate_roic, 
                                      calculate_interest_burden_ratio, calculate_interest_coverage_ratio, 
                                      calculate_leverage_ratio, calculate_asset_turnover_ratio, 
                                      calculate_fixed_asset_turnover_ratio, calculate_inventory_turnover_ratio, 
                                      calculate_receivables_turnover_ratio, calculate_payables_turnover_ratio, 
                                      calculate_current_ratio, calculate_quick_ratio, calculate_cash_ratio, 
                                      calculate_gross_profit_margin, calculate_ebitda_margin, calculate_ebit_margin)

In [52]:
# Download the stock data

# Define the ticker for Viva Energy Group Ltd
# (ASX: VEA)
TICKER = "VEA.AX"

try:
    # Get stock information
    stock = yf.Ticker(TICKER)
    stock_prices = stock.history(period="10y")
    stock_info = stock.info
    stock_financials = stock.financials
    stock_balance_sheet = stock.balance_sheet
    stock_cashflow = stock.cashflow

    # Extract relevant information
    last_price = stock_prices['Close'].iloc[-1]
    net_income = stock_financials.loc["Net Income"].iloc[0]
    revenue = stock_financials.loc["Total Revenue"].iloc[0]
    dividends_paid = stock_info['lastDividendValue'] * stock_info['sharesOutstanding']  
    shares_outstanding = stock_info['sharesOutstanding']
    eps = net_income / shares_outstanding
    total_assets = (stock_balance_sheet.loc["Total Assets"].iloc[0] + stock_balance_sheet.loc["Total Assets"].iloc[1]) / 2
    total_equity = stock_balance_sheet.loc["Stockholders Equity"].iloc[0]

    # Free cash flow
    free_cash_flow = stock_cashflow.loc['Free Cash Flow'].iloc[0:4]

    # Metrics needed for other ratios
    invested_capital = total_assets - stock_balance_sheet.loc["Current Liabilities"].iloc[0]
    ebit = stock_financials.loc["EBIT"].iloc[0]
    interest_expense = stock_financials.loc["Interest Expense"].iloc[0]
    net_fixed_assets = stock_balance_sheet.loc["Net PPE"].iloc[0]
    cost_of_goods_sold = stock_financials.loc["Cost Of Revenue"].iloc[0]

    # For turnover ratios (average balances)
    average_inventory = stock_balance_sheet.loc["Inventory"].iloc[0]  
    average_accounts_receivable = stock_balance_sheet.loc["Accounts Receivable"].iloc[0]
    average_accounts_payable = stock_balance_sheet.loc["Accounts Payable"].iloc[0]

    # For cash conversion cycle (assume 365 days/year for turnover days)
    inventory_days = (average_inventory / cost_of_goods_sold) * 365
    receivables_days = (average_accounts_receivable / revenue) * 365
    payables_days = (average_accounts_payable / cost_of_goods_sold) * 365

    # Liquidity ratios
    current_assets = stock_balance_sheet.loc["Current Assets"].iloc[0]
    current_liabilities = stock_balance_sheet.loc["Current Liabilities"].iloc[0]
    inventory = stock_balance_sheet.loc["Inventory"].iloc[0]
    cash_and_cash_equivalents = stock_balance_sheet.loc["Cash And Cash Equivalents"].iloc[0]

    # Margins
    gross_profit = stock_financials.loc["Gross Profit"].iloc[0]
    ebitda = stock_financials.loc["EBITDA"].iloc[0]

    print(f"Chosen Stock: {TICKER} \nLast Share Price: {last_price:.2f} \nTotal Assets: {total_assets}")

except Exception as e:
    print(f"An error occurred: {e}")

Chosen Stock: VEA.AX 
Last Share Price: 1.65 
Total Assets: 10603800000.0


In [53]:
# Calculate financial ratios and metrics using functions from Financial_Ratios_Metrics.py

# Creating a dictionary with ratio names as keys and their functions as values (to calculate key ratios using imported functions)
ratios = {
    "Net Profit Margin": calculate_net_profit_margin(net_income, revenue),
    "EPS": calculate_eps(net_income, shares_outstanding),
    "P/E Ratio": calculate_pe_ratio(last_price, eps),
    "DPS": calculate_dividends_per_share(dividends_paid, shares_outstanding),
    "DPR": calculate_dividend_payout_ratio(dividends_paid, net_income),
    "ROE": calculate_roe(net_income, total_equity),
    "ROA": calculate_roa(net_income, total_assets),
    "ROIC": calculate_roic(net_income, invested_capital),
    "Interest Burden": calculate_interest_burden_ratio(ebit, interest_expense),
    "Interest Coverage": calculate_interest_coverage_ratio(ebit, interest_expense),
    "Leverage": calculate_leverage_ratio(total_assets, total_equity),
    "Asset Turnover": calculate_asset_turnover_ratio(revenue, total_assets),
    "Fixed Asset Turnover": calculate_fixed_asset_turnover_ratio(revenue, net_fixed_assets),
    "Inventory Turnover": calculate_inventory_turnover_ratio(cost_of_goods_sold, average_inventory),
    "Receivables Turnover": calculate_receivables_turnover_ratio(revenue, average_accounts_receivable),
    "Payables Turnover": calculate_payables_turnover_ratio(cost_of_goods_sold, average_accounts_payable),
    "Current Ratio": calculate_current_ratio(current_assets, current_liabilities),
    "Quick Ratio": calculate_quick_ratio(current_assets, current_liabilities, inventory),
    "Cash Ratio": calculate_cash_ratio(cash_and_cash_equivalents, current_liabilities),
    "Gross Profit Margin": calculate_gross_profit_margin(gross_profit, revenue),
    "EBITDA Margin": calculate_ebitda_margin(ebitda, revenue),
    "EBIT Margin": calculate_ebit_margin(ebit, revenue)
}

# Loop through the dictionary and print each ratio with its value
for ratio_name, ratio_value in ratios.items():
    # Format ratios with both real numbers and percentages to two decimal places
    print(f"{ratio_name}: {ratio_value:.2f}" if isinstance(ratio_value, float) else f"{ratio_name}: {ratio_value:.2%}")


Net Profit Margin: -0.00
EPS: -0.05
P/E Ratio: -34.92
DPS: 0.04
DPR: -0.82
ROE: -0.04
ROA: -0.01
ROIC: -0.01
Interest Burden: -1.98
Interest Coverage: 0.34
Leverage: 5.59
Asset Turnover: 2.83
Fixed Asset Turnover: 5.28
Inventory Turnover: 13.00
Receivables Turnover: 18.56
Payables Turnover: 16.63
Current Ratio: 0.95
Quick Ratio: 0.51
Cash Ratio: 0.04
Gross Profit Margin: 0.10
EBITDA Margin: 0.02
EBIT Margin: 0.00


In [54]:
# Set up fiscal years and corresponding index positions
years = ["FY21", "FY22", "FY23", "FY24"]
indices = [3, 2, 1, 0]  # Assuming iloc[3] = FY21 and iloc[0] = FY24

# Create an empty dictionary
financial_summary_data = {}

# Loop through each year and collect metrics
for year, idx in zip(years, indices):
    financial_summary_data[year] = {
        "Revenue": stock_financials.loc["Total Revenue"].iloc[idx],
        "EBITDA": stock_financials.loc["EBITDA"].iloc[idx],
        "EBIT": stock_cashflow.loc["Free Cash Flow"].iloc[idx],
        "Net Profit": stock_financials.loc["Net Income"].iloc[idx],
        "Profit Margin": calculate_net_profit_margin(
            stock_financials.loc["Net Income"].iloc[idx],
            stock_financials.loc["Total Revenue"].iloc[idx]
        ),
        "EPS": (net_income / shares_outstanding),
        "DPS": stock_info['lastDividendValue'],  # Assuming DPS stays flat, otherwise need yearly logic
        "DPR": calculate_dividend_payout_ratio(
            stock_info['lastDividendValue'] * stock_info['sharesOutstanding'],
            stock_financials.loc["Net Income"].iloc[idx]
        ),
        "ROE": calculate_roe(
            stock_financials.loc["Net Income"].iloc[idx],
            stock_balance_sheet.loc["Common Stock Equity"].iloc[idx]
        ),
        "ROA": calculate_roa(
            stock_financials.loc["Net Income"].iloc[idx],
            stock_balance_sheet.loc["Total Assets"].iloc[idx]
        ),
        "ROIC": calculate_roic(
            stock_financials.loc["Net Income"].iloc[idx],
            stock_balance_sheet.loc["Invested Capital"].iloc[idx]
        )
    }

# Create DataFrame
financial_summary_df = pd.DataFrame(financial_summary_data)

# Round to 2 decimal places
financial_summary_df = financial_summary_df.round(2)

# Display
print("Financial Summary - Viva Energy Group Ltd (ASX: VEA):")
print(financial_summary_df)


Financial Summary - Viva Energy Group Ltd (ASX: VEA):
                       FY21          FY22          FY23          FY24
Revenue        1.590000e+10  2.619650e+10  2.561540e+10  2.999220e+10
EBITDA         7.559000e+08  1.172800e+09  5.624000e+08  6.253000e+08
EBIT           3.447000e+08  8.023000e+08  1.863000e+08  1.750000e+07
Net Profit     2.329000e+08  5.143000e+08  3.800000e+06 -7.630000e+07
Profit Margin  1.000000e-02  2.000000e-02  0.000000e+00 -0.000000e+00
EPS           -5.000000e-02 -5.000000e-02 -5.000000e-02 -5.000000e-02
DPS            4.000000e-02  4.000000e-02  4.000000e-02  4.000000e-02
DPR            2.700000e-01  1.200000e-01  1.645000e+01 -8.200000e-01
ROE            1.100000e-01  2.200000e-01  0.000000e+00 -4.000000e-02
ROA            3.000000e-02  6.000000e-02  0.000000e+00 -1.000000e-02
ROIC           1.000000e-01  2.200000e-01  0.000000e+00 -2.000000e-02


In [55]:
# Create the "Ratio Analysis" table

# Set up fiscal years and corresponding index positions
years = ["FY21", "FY22", "FY23", "FY24"]
indices = [3, 2, 1, 0]  # Assuming iloc[3] = FY21, iloc[0] = FY24

# Create a dictionary to store  ratios
ratio_analysis_data = {}

# Create a loop for each year and calculate all ratios
for year, idx in zip(years, indices):
    ratio_analysis_data[year] = {
        # Leverage Ratios
        "Interest Burden": calculate_interest_burden_ratio(stock_financials.loc["EBIT"].iloc[idx], stock_financials.loc["Interest Expense"].iloc[idx]),
        "Interest Coverage": calculate_interest_coverage_ratio(stock_financials.loc["EBIT"].iloc[idx], stock_financials.loc["Interest Expense"].iloc[idx]),
        "Leverage": calculate_leverage_ratio(stock_balance_sheet.loc["Total Assets"].iloc[idx], stock_balance_sheet.loc["Common Stock Equity"].iloc[idx]),

        # Asset Utilisation Ratios
        "Total Asset Turnover": calculate_asset_turnover_ratio(stock_financials.loc["Total Revenue"].iloc[idx], stock_balance_sheet.loc["Total Assets"].iloc[idx]),
        "Fixed Asset Turnover": calculate_fixed_asset_turnover_ratio(stock_financials.loc["Total Revenue"].iloc[idx], stock_balance_sheet.loc["Net PPE"].iloc[idx]),
        "Inventory Turnover": calculate_inventory_turnover_ratio(stock_financials.loc["Cost Of Revenue"].iloc[idx], stock_balance_sheet.loc["Inventory"].iloc[idx]),
        "Receivables Turnover": calculate_receivables_turnover_ratio(stock_financials.loc["Total Revenue"].iloc[idx], stock_balance_sheet.loc["Accounts Receivable"].iloc[idx]),
        "Payables Turnover": calculate_payables_turnover_ratio(stock_financials.loc["Cost Of Revenue"].iloc[idx], stock_balance_sheet.loc["Accounts Payable"].iloc[idx]),

        # Liquidity Ratios
        "Current Ratio": calculate_current_ratio(stock_balance_sheet.loc["Current Assets"].iloc[idx], stock_balance_sheet.loc["Current Liabilities"].iloc[idx]),
        "Quick Ratio": calculate_quick_ratio(stock_balance_sheet.loc["Current Assets"].iloc[idx], stock_balance_sheet.loc["Current Liabilities"].iloc[idx], stock_balance_sheet.loc["Inventory"].iloc[idx]),
        "Cash Ratio": calculate_cash_ratio(stock_balance_sheet.loc["Cash And Cash Equivalents"].iloc[idx], stock_balance_sheet.loc["Current Liabilities"].iloc[idx]),

        # Profitability Ratios
        "Return on Assets (ROA)": calculate_roa(stock_financials.loc["Net Income"].iloc[idx], stock_balance_sheet.loc["Total Assets"].iloc[idx]),
        "Return on Equity (ROE)": calculate_roe(stock_financials.loc["Net Income"].iloc[idx], stock_balance_sheet.loc["Common Stock Equity"].iloc[idx]),
        "Return on Invested Capital (ROIC)": calculate_roic(stock_financials.loc["Net Income"].iloc[idx], stock_balance_sheet.loc["Invested Capital"].iloc[idx]),

        # Margin Ratios
        "Gross Profit Margin": calculate_gross_profit_margin(stock_financials.loc["Gross Profit"].iloc[idx], stock_financials.loc["Total Revenue"].iloc[idx]),
        "EBITDA Margin": calculate_ebitda_margin(stock_financials.loc["EBITDA"].iloc[idx], stock_financials.loc["Total Revenue"].iloc[idx]),
        "EBIT Margin": calculate_ebit_margin(stock_financials.loc["EBIT"].iloc[idx], stock_financials.loc["Total Revenue"].iloc[idx]),
        "Net Profit Margin": calculate_net_profit_margin(stock_financials.loc["Net Income"].iloc[idx], stock_financials.loc["Total Revenue"].iloc[idx])
    }

# Create DataFrame
ratio_analysis_df = pd.DataFrame(ratio_analysis_data)

# Display
print("Ratio Analysis - Viva Energy Group Ltd (ASX: VEA):")
print(ratio_analysis_df)


Ratio Analysis - Viva Energy Group Ltd (ASX: VEA):
                                        FY21       FY22       FY23       FY24
Interest Burden                     0.950720   0.952121   0.310491  -1.977612
Interest Coverage                  20.292135  20.885870   1.450307   0.335840
Leverage                            3.460817   3.652830   4.517209   6.397225
Total Asset Turnover                2.175430   3.037768   2.820365   2.473522
Fixed Asset Turnover                4.294744   7.019050   6.315901   5.278273
Inventory Turnover                 11.994404  15.382630  13.565239  13.003318
Receivables Turnover               13.740062  15.280273  15.950806  18.564125
Payables Turnover                  17.541723  21.619318  19.781265  16.626722
Current Ratio                       1.050181   1.037450   1.001116   0.949390
Quick Ratio                         0.574768   0.620992   0.564751   0.506422
Cash Ratio                          0.038976   0.077487   0.052301   0.041046
Return on Ass